<a href="https://colab.research.google.com/github/avrland/dataScienceFromScratch/blob/main/_Niusbot_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  #NiusBot 2021

Na podstawie:
https://minimaxir.com/2018/05/text-neural-networks/


In [ ]:
%tensorflow_version 1.x
!pip install -q textgenrnn
from google.colab import files
from textgenrnn import textgenrnn
from datetime import datetime
import os

TensorFlow 1.x selected.


Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
%tensorflow_version 1.x
!pip install -q textgenrnn
from textgenrnn import textgenrnn
#Montowanie Google Drive
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

# Pobieranie bazy tytułów źródłowych
Pobieramy bazę i odczytujemy kilka pierwszych linijek, aby sprawdzić czy wszystko zostało pobrane poprawnie.

In [ ]:
!pip install wget
import wget
url = 'https://raw.githubusercontent.com/avrland/dataScienceFromScratch/main/titlesBase/titlesPL_6k.txt'
wget.download(url, '/content/titles.txt')
filename = '/content/titles.txt'  
with open(filename) as fn:  
  ln = fn.readline()
  lncnt = 0
  while lncnt < 5:
       print("Line {}: {}".format(lncnt, ln.strip()))
       ln = fn.readline()
       lncnt += 1

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=d2a2abd26cdda26b07d2d42cbd1fb0ad1e2a6b777b28b8b6550a7f28630d4dc9
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
Line 0: 
Line 1: Polka straciła 36 tys. zł: napastnik wykiwał zarówno ją, jak i bank
Line 2: Chrome 86 na Androida pozwoli zaplanować pobieranie. Można już testować
Line 3: Poczta Polska i cyfrowa rewolucja. Identyfikacja RFID przyspieszy wysyłki
Line 4: GOG GALAXY 2.0 łączy siły z Epic Games Store. Jest wreszcie oficjalna integracja


# Parametry modelu i trenowania
Set the textgenrnn model configuration here: the default parameters here give good results for most workflows. (see the [demo notebook](https://github.com/minimaxir/textgenrnn/blob/master/docs/textgenrnn-demo.ipynb) for more information about these parameters)

If you are using an input file where documents are line-delimited, make sure to set `line_delimited` to `True`.

In [ ]:
model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 512,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 5,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': True,   # consider text both forwards and backward, can give a training boost
    'max_length': 10,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': True,   # set to True if each text has its own line in the source file
    'num_epochs': 8,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': True,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [ ]:
textgen.generate(10)

" " the umbrella academy . " nie ma mocnych na ukrainie

" to nie mogło się na dolnym śląsku . "

" " se " . " nie ma być przestrzegane nowak . 25 wyzwań "

" to nie ma czegoś takiego " . "

" rolnik szuka karty . " byliśmy spadł z

" " : nie ma czegoś takiego nie ma możliwości z twarzą

" : w sprawie białorusi . są ofiary

" to nie się na białorusi . jest terminarz

" " " . . " the outer worlds

" to polityki " . nie ma ponad 300



The next cell will start the actual training. And thanks to the power of Keras's CuDNN layers, training is super-fast when compared to CPU training on a local machine!

Ideally, you want a training loss less than `1.0` in order for the model to create sensible text consistently.

In [ ]:
import shutil
file_name = '/content/titles.txt'

#Pobierz dzisiejszą datę i godzinę
from datetime import datetime
timeNow = str(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

model_name = 'niusbot_' + timeNow   # change to set file name of resulting trained models/texts

textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=512,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

shutil.copy('{}_weights.hdf5'.format(model_name), '/content/drive/My Drive/NIUSBOT/noweModele')
shutil.copy('{}_vocab.json'.format(model_name), '/content/drive/My Drive/NIUSBOT/noweModele')
shutil.copy('{}_config.json'.format(model_name), '/content/drive/My Drive/NIUSBOT/noweModele')

6,151 texts collected.
Training new model w/ 5-layer, 512-cell Bidirectional LSTMs
Training on 60,425 word sequences.
Epoch 1/8
118/118 [==============================] - 32s 274ms/step - loss: 34.2065 - val_loss: 8.3542
Epoch 2/8
118/118 [==============================] - 26s 219ms/step - loss: 6.8845 - val_loss: 7.3707
Epoch 3/8
118/118 [==============================] - 26s 218ms/step - loss: 6.0600 - val_loss: 7.4040
Epoch 4/8
118/118 [==============================] - 26s 218ms/step - loss: 5.5487 - val_loss: 7.2387
Epoch 5/8
118/118 [==============================] - 26s 219ms/step - loss: 5.0402 - val_loss: 7.6485
####################
Temperature: 0.2
####################
" : nie ma nie ma nie ma z

" : nie ma z koronawirusem

" : nie ma nie ma z

####################
Temperature: 0.5
####################
" : nie ma dzieje się . "

koronawirus w polsce . " : nie się o koronawirusie

" to nie ma z koronawirusem

####################
Temperature: 1.0
####################
crusader 

'/content/drive/My Drive/NIUSBOT/noweModele/niusbot_2021-03-24 09:20:37_config.json'

You can download the weights and configuration files in the cell below, allowing you recreate the model on your own computer!

In [ ]:
files.download('{}_weights.hdf5'.format(model_name))
files.download('{}_vocab.json'.format(model_name))
files.download('{}_config.json'.format(model_name))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Generowanie z gotowego modelu z pliku

In [ ]:
%tensorflow_version 1.x
!pip install -q textgenrnn
from textgenrnn import textgenrnn
#Montowanie Google Drive
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

modelPath = "/content/drive/MyDrive/NIUSBOT/titles"
textgen = textgenrnn(weights_path=modelPath + '_weights.hdf5',
                       vocab_path=modelPath + '_vocab.json',
                       config_path=modelPath + '_config.json')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
textgen.generate(10, temperature=1.0)

xbox series s istnieje

: decyzje ws . podwyżek płac : pozycja

polacy solidarni z białorusinami . podano przyczynę zgonu

pierwszy przypadek koronawirusa w kadrze lekkoatletek

zarzuty dla na plaży . ea nim " . . .

koronawirus w polsce : 575 nowych przypadków ; zmarła jedna osoba [ zdjęcia ]

na drodze . " teraz już teraz się , kiedy je "

marvel ' s avengers – zobacz bohaterów w akcji na steamie

na ceny gier na plaży . grali nad barierką

staną przed sądem za złota . producent przypomina 2014 r .

